In [1]:
import datetime
from pathlib import Path
import os
import cv2
import numpy as np
import matplotlib.colors as mcolors
import PySimpleGUI as sg
import re

def get_YYYYMMDDhhmmss():
    
    t_delta = datetime.timedelta(hours=9)
    JST = datetime.timezone(t_delta, 'JST')
    now = datetime.datetime.now(JST)

    d = now.strftime('%Y%m%d%H%M%S')
    return d
    
def get_color_code(r,g,b):
    color_code = '#{}{}{}'.format(get_hex_00pad(r),get_hex_00pad(g),get_hex_00pad(b))
    return color_code

def get_hex_00pad(value):
    return hex(value).replace('0x', '').zfill(2)
def get_target_pxl_indices(original_img,trg_clr_cd,clr_range):    
    
    trg_clr_rgb = tuple(int(c*255) for c in mcolors.to_rgb(trg_clr_cd))
    trg_clr_hsv = get_hsv(trg_clr_rgb[0],trg_clr_rgb[1],trg_clr_rgb[2])
    
    #色相
    hbase=trg_clr_hsv[0]
    #許容量の上限・下限
    hupper=clr_range
    hlower=clr_range
    if hlower<0:
        hlower = 0
    #H値の範囲を指定
    hmax=hbase+hupper
    hmin=hbase-hlower
    
    imghsv=cv2.cvtColor(original_img, cv2.COLOR_BGR2HSV)
    np_arr_h=imghsv[:,:,0] 
    
    #色相を変更するピクセルのインデックスを配列で取得
    #trg_pxl_arrを参照して、H・S・Vそれぞれを変更
    trg_pxl_arr = np.where((np_arr_h<hmax) & (np_arr_h>hmin))

    return trg_pxl_arr;
    
def get_cleared_color_img(original_img,trg_clr_cd,clr_range):
    
    trg_pxl_arr = get_target_pxl_indices(original_img,trg_clr_cd,clr_range)
    
    #αチャンネルありに変換
    imgrgba = cv2.cvtColor(original_img, cv2.COLOR_RGB2RGBA)
    
    np_arr_alf=imgrgba[:,:,3]
    np_arr_alf[trg_pxl_arr[0],trg_pxl_arr[1]] = 0 #透過
    imgrgba[:,:,3]=np_arr_alf   
    
    return imgrgba
    
def get_replaced_color_img(original_img,trg_clr_cd,replace_clr_cd,clr_range):
    
    replace_clr_rgb = tuple(int(c*255) for c in mcolors.to_rgb(replace_clr_cd))
    replace_clr_hsv = get_hsv(replace_clr_rgb[0],replace_clr_rgb[1],replace_clr_rgb[2])

    trg_pxl_arr = get_target_pxl_indices(original_img,trg_clr_cd,clr_range)
    
    #合致するピクセルの色相を変更
    imghsv=cv2.cvtColor(original_img, cv2.COLOR_BGR2HSV)
    np_arr_h=imghsv[:,:,0] 
    
    np_arr_h[trg_pxl_arr[0],trg_pxl_arr[1]] = replace_clr_hsv[0]    
    imghsv[:,:,0]=np_arr_h   

    #彩度(S)
    np_arr_s=imghsv[:,:,1]
    np_arr_s[trg_pxl_arr[0],trg_pxl_arr[1]] = replace_clr_hsv[1]     
    imghsv[:,:,1]=np_arr_s
    
    #明度(V)
    np_arr_v=imghsv[:,:,2]
    #v=np.where(v == vbase,vdelta,0)
    np_arr_v[trg_pxl_arr[0],trg_pxl_arr[1]] = replace_clr_hsv[2]      
    imghsv[:,:,2]=np_arr_v
    
    result=cv2.cvtColor(imghsv, cv2.COLOR_HSV2BGR)
    
    return result

def get_hsv(r, g, b):
    hsv = cv2.cvtColor(np.array([[[b, g, r]]], dtype=np.uint8), cv2.COLOR_BGR2HSV)[0][0]
    return (hsv[0], hsv[1], hsv[2])
#日本語対策
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
def color_pick():
    widget = window["-orginal_img-"].Widget
    x = widget.winfo_pointerx() - widget.winfo_rootx()
    y = widget.winfo_pointery() - widget.winfo_rooty()
    
    tmp_bytes = widget.image.cget('data')
    img_np_arr = np.frombuffer(tmp_bytes, dtype=np.uint8)
    img = cv2.imdecode(img_np_arr, flags=1)
    list = img[y,x].tolist()
    #引数はRGBの順にする
    color_code = get_color_code(list[2],list[1],list[0])

    window['-target_color_code-'].update(color_code)
    window['-target_color-'].update(background_color=color_code)
    
pattern = '^#([\da-fA-F]{6}|[\da-fA-F]{3})$'

original_img = [[sg.Image(filename='', key='-orginal_img-', enable_events=True)]]
modify_img = [[sg.Image(filename='', key='-modify_img-', enable_events=True)]]

read_file = [[sg.Text("ファイル"), sg.InputText(key='-input_file-', enable_events=True, ),
                sg.FileBrowse('選択', key='-file-', target="-input_file-",  file_types = (("画像ファイル", ".jpg"), ("画像ファイル", ".png"))), sg.Button('読込', key='-read_file-')]]

color_picker1 = [[sg.Text('',size=(3,1),background_color='#FFFFFF',key='-target_color-'),sg.InputText(key='-target_color_code-',size=(8,1),enable_events=True,disabled=False), sg.ColorChooserButton('色の選択', target='-target_color_code-',key='-color_btn-')],
                 [sg.Text('画像をクリックすると色を選択できます')]]
color_picker2 = [[sg.Text('',size=(3,1),background_color='#FFFFFF',key='-replace_color-'),sg.InputText("#FFFFFF",key='-replace_color_code-',size=(8,1),enable_events=True,disabled=False), sg.ColorChooserButton('色の選択', target='-replace_color_code-',key='-color_btn2-')]]

#許容量は初期値5にしておく(0だと対象となるピクセルが少ないため)
slider = [[sg.Slider((0, 255), 5, 1, orientation='h',key='-color_range_slider-',
                      enable_events=True),
                sg.InputText(default_text='5',size=(5,1),key='-color_range-')]]

row1_col1 = [
    [sg.Frame(title='ファイルの選択・読込',layout=read_file)]
]

row1_col2 =[
    [sg.Frame(title='置換する色',layout=color_picker1)],
    [sg.Frame(title='置換後の色',layout=color_picker2)]
]

row1_col3 =[
    [sg.Frame(title='許容量',layout=slider)],
    [sg.Button('透過',key='-maketrans-'),sg.Button('置換',key='-replace-')],
    [sg.Button('保存',key='-save-'),sg.Button('リセット',key='-reset-')]
]

row2_col1 =[
    [sg.Frame(title='オリジナル',layout=original_img)]
]

row2_col2 =[
    [sg.Frame(title='処理結果',layout=modify_img)]
]

layout_tot = [
    [sg.Column(row1_col1,vertical_alignment='t'), sg.Column(row1_col2,key="row1_col2",visible=False), sg.Column(row1_col3,key="row1_col3",visible=False)],
    [sg.Column(row2_col1,key="row2_col1", size=(500, 500), scrollable=True,visible=False),sg.Column(row2_col2,key="row2_col2", size=(500, 500), scrollable=True,visible=False)]
    
]    

# flag
READ_File = False

sg.theme('LightGray1')

window = sg.Window('色置換・削除ツール', layout_tot,
                   location=(10, 10),alpha_channel=1.0,
                   no_titlebar=False,grab_anywhere=False).Finalize()

while True:

    event, values = window.read(timeout=20)
    #キャンセルボタンは使用していないので、現状では下記２行が不要
    if event in (None, 'Cancel'):
        break
    elif event == '-read_file-':
        if values['-input_file-'] != '':
            window["row1_col2"].update(visible=True)
            window["row1_col3"].update(visible=True)
            window["row2_col1"].update(visible=True)
            window["row2_col2"].update(visible=True)
            read_path = Path(values['-input_file-'])
            #漢字含むパスだとエラーなので回避用の関数をかませる
            read_img = imread(str(read_path))
            imgbytes = cv2.imencode('.png', read_img)[1].tobytes()
            mod_img = read_img.copy()
            window['-orginal_img-'].update(data=imgbytes)
            window['-modify_img-'].update(data=imgbytes)
            window.refresh()
            window['row2_col1'].contents_changed()
            window['row2_col2'].contents_changed()
            READ_File = True
    
    if READ_File:
        if event == "-color_range_slider-":
            window['-color_range-']. Update(int(values['-color_range_slider-']))
        elif event.endswith('_code-'):
            print("done")
            color = values[event]
            if color != 'None':
                result = re.match(pattern, color)
                str_event = event.replace("_code","")
                if result:
                    window[str_event].update(background_color=color)
                else:
                    window[str_event].update(background_color='#FFFFFF')
        elif event == ('-orginal_img-'):
            color_pick()
        elif event == ('-maketrans-'):
            target_color = values['-target_color_code-']
            if re.match(pattern, target_color):
                color_range = int(values['-color_range-'])
                cleared_img = get_cleared_color_img(read_img,target_color,color_range)
                mod_imgbytes = cv2.imencode('.png', cleared_img)[1].tobytes()
                mod_img = cleared_img.copy()
                window['-modify_img-'].update(data=mod_imgbytes)
            else:
                window['-target_color_code-'].update('')
        elif event == ('-replace-'):
            target_color = values['-target_color_code-']
            replace_color = values['-replace_color_code-']
            if re.match(pattern, target_color) and re.match(pattern, replace_color):
                color_range = int(values['-color_range-'])
                replaced_img = get_replaced_color_img(read_img,target_color,replace_color,color_range)
                mod_imgbytes = cv2.imencode('.png', replaced_img)[1].tobytes()
                mod_img = replaced_img.copy()
                window['-modify_img-'].update(data=mod_imgbytes)
            else:
                window['-target_color_code-'].update('')
                window['-replace_color_code-'].update('')
        elif event == '-reset-':
            mod_img = read_img.copy()
            window['-modify_img-'].update(data=imgbytes)
            window.refresh()
            window['row2_col2'].contents_changed()               
        elif event == '-save-':
            out_path = "tmp_" + get_YYYYMMDDhhmmss() + ".png"
            cv2.imwrite(out_path, mod_img)
            #cv2が日本語未対応のため、一度仮名でファイル出力し、すぐにリネーム
            os.rename(out_path,f'{read_path.stem}_{get_YYYYMMDDhhmmss()}.png')

window.close()
